In [3]:
# Hello there.   This is a do-over of the floating preferred analyzer

# Time to develop clean modules. Let's start with reading in a list of prefs
# The input data consists of three separate tables:
# a) A listing of tickers and their configurations
# b) A list of ratings for each company (equivalently, of the ticker)
# c) The current interest rate environment, which has only the 3-month rate and the prime rate



In [7]:
# Imports go here
import pandas as pd
import requests_cache
# local ones
import prefcode as pc


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Set up cache
session_cached = requests_cache.CachedSession('yfinance.cache',expire_after=3600)
session_uncached = requests_cache.CachedSession('yfinance2.cache',expire_after=30)


In [22]:
# Read the prefs database

print("Setting up databases")
df, interest_db = pc.setup_databases()

print("Fetching price data")
df = pc.fetch_prices(df, session_cached, fetch=True)

print("Updating dividends and current yield")
df = pc.update_div_and_yield(df, interest_db,price_column="Price")

print("Updating market spread for later calculation")
df2 = pc.update_market_spread(df, interest_db)

print("Dropping prime-related preferreds. Maybe another day")
tdf = df[df['Type'] == 'T'].copy()

print("Calculating scenarios")
scenarios = {"010":  [0.10,  0.05],
             "018":  [0.18,  0.25],
             "030":  [0.30,  0.50],
             "050":  [0.50,  0.20]}

pc.update_expected_yield(tdf, scenarios)


Setting up databases
Fetching price data
Updating dividends and current yield
Updating market spread for later calculation
Dropping prime-related preferreds. Maybe another day
Calculating scenarios


In [28]:
print("Here are the highest yield items")
tdf.sort_values(by='ExpYield', ascending=False).head(5)


Here are the highest yield items


,Ticker,Spread,Type,Mult,Rating,Price,AnnualDiv,CurYieldPct,MSpread,010_Yield,018_Yield,030_Yield,050_Yield,ExpYield
22,FN.PR.B,207.0,T,NaN,P3I,13.32,0.5632,4.2282,4.0452,2.4822,4.1755,6.5986,10.3528,6.537845
33,TRP.PR.H,128.0,T,NaN,P2L,12.85,0.3658,2.8467,2.6637,-0.0073,2.7330,6.5660,12.3082,6.427525
30,TA.PR.E,203.0,T,NaN,P3L,13.68,0.5532,4.0439,3.8609,2.3181,3.9905,6.3785,10.0664,6.316060
15,CVE.PR.B,173.0,T,NaN,P2L,14.26,0.4783,3.3541,3.1711,1.4335,3.2774,5.8838,9.8491,5.802745
34,TRP.PR.I,154.0,T,NaN,P2L,14.35,0.4308,3.0021,2.8191,0.8796,2.9159,5.7745,10.0795,5.676105
19,FFH.PR.F,216.0,T,NaN,P3H,16.06,0.5857,3.6469,3.4639,2.3606,3.6110,5.3869,8.1078,5.335790
20,FFH.PR.H,256.0,T,NaN,P3H,17.10,0.6857,4.0099,3.8269,3.0420,3.9837,5.3278,7.4023,5.292385
12,BPO.PR.S,348.0,T,NaN,P3I,19.75,0.9157,4.6365,4.4535,4.1565,4.6238,5.2951,6.3418,5.279685
23,FTS.PR.I,145.0,T,NaN,P3H,15.35,0.4083,2.6599,2.4769,0.6238,2.5780,5.2982,9.3446,5.193710
28,SLF.PR.J,141.0,T,NaN,P2H,15.42,0.3982,2.5824,2.3994,0.5306,2.5211,5.2857,9.3847,5.176595


In [29]:
best_scn_df = pc.summarize_best_by_column(tdf, 'ExpYield')
best_scn_df = best_scn_df.reindex(columns=["Ticker", "Rating", "Spread","ExpYield"])

print("And here are the best by rating level")
best_scn_df.sort_values(by=['Rating'])

And here are the best by rating level


,Ticker,Rating,Spread,ExpYield
28,SLF.PR.J,P2H,141.0,5.176595
25,MFC.PR.P,P2I,141.0,4.599385
33,TRP.PR.H,P2L,128.0,6.427525
19,FFH.PR.F,P3H,216.0,5.335790
22,FN.PR.B,P3I,207.0,6.537845
30,TA.PR.E,P3L,203.0,6.316060


,Ticker,Spread,Type,Mult,Rating,Price,AnnualDiv,CurYieldPct,MSpread,Scn1_Div
26,PWF.PR.A,NaN,P,0.7,P2H,14.41,0.4288,2.9757,0.5257,NaN
24,IFC.PR.D,266.0,T,NaN,P2I,24.87,0.7107,2.8577,2.6747,0.7650
3,BAM.PR.E,NaN,P,1.0,P2L,17.35,0.6125,3.5303,1.0803,NaN
21,FFH.PR.J,285.0,T,NaN,P3H,18.61,0.7582,4.0742,3.8912,0.8125
12,BPO.PR.S,348.0,T,NaN,P3I,19.75,0.9157,4.6365,4.4535,0.9700
30,TA.PR.E,203.0,T,NaN,P3L,13.68,0.5532,4.0439,3.8609,0.6075


In [251]:
# Now, calculate market spread, which is the rate above the benchmark
# required by the market. 


                     
# df.sort_values(by=['MSpread'])

In [364]:


#tdf.drop(columns='013_Yield', inplace=True, errors='ignore')

best_scn_df = pc.summarize_best_by_column(tdf, 'ExpYield')
# foo = best_scn_df[best_scn_df['Rating'] == 'P2H']


best_scn_df = best_scn_df.reindex(columns=["Ticker", "Rating", "Spread","ExpYield"])
best_scn_df.sort_values(by=['Rating'])

,Ticker,Rating,Spread,ExpYield
28,SLF.PR.J,P2H,141.0,5.176595
25,MFC.PR.P,P2I,141.0,4.599385
33,TRP.PR.H,P2L,128.0,6.427525
19,FFH.PR.F,P3H,216.0,5.335790
22,FN.PR.B,P3I,207.0,6.537845
30,TA.PR.E,P3L,203.0,6.316060


In [358]:


best_df = pc.summarize_best_by_column(tdf, 'ExpYield')

#make_portfolio_recommendation(tdf, 'P3H')
for i in best_df.index:
    rating = best_df.at[i,"Rating"]
    pc.make_portfolio_recommendation(tdf, rating, scenarios)
    


Rating:  P3I
Allocate to only:  FN.PR.B 6.13572  >  1.998795
Rating:  P3H
Allocate to only:  FTS.PR.I 5.04763  >  4.1424565
Rating:  P2I
Allocate to only:  MFC.PR.P 4.4569849999999995  >  1.35977675
ERROR! Session/line number was not unique inRating:  P2H
 Allocate to only:  SLF.PR.J 5.042085  >  1.5096957500000001
database. History logging moved to new session Rating: 195
 P3L
Allocate to only:  TA.PR.E 5.937659999999999  >  1.9176805
Rating:  P2L
Allocate to only:  TRP.PR.H 6.35932  >  2.0355119999999998


In [341]:
best_scn_df = summarize_best_by_column(tdf, 'ExpYield')
best_scn_df.sort_values(by=['Rating'])

,Ticker,Spread,Type,Mult,Rating,Price,AnnualDiv,CurYieldPct,MSpread,030_Yield,050_Yield,010_Yield,ExpYield
28,SLF.PR.J,141.0,T,NaN,P2H,15.42,0.3982,2.5824,2.3994,5.2857,9.3847,0.5306,5.042085
25,MFC.PR.P,141.0,T,NaN,P2I,16.75,0.3982,2.3773,2.1943,4.7002,8.1839,0.6091,4.456985
33,TRP.PR.H,128.0,T,NaN,P2L,12.85,0.3658,2.8467,2.6637,6.5660,12.3082,-0.0073,6.359320
23,FTS.PR.I,145.0,T,NaN,P3H,15.35,0.4083,2.6599,2.4769,5.2982,9.3446,0.6238,5.047630
22,FN.PR.B,207.0,T,NaN,P3I,13.32,0.5632,4.2282,4.0452,6.5986,10.3528,2.4822,6.135720
30,TA.PR.E,203.0,T,NaN,P3L,13.68,0.5532,4.0439,3.8609,6.3785,10.0664,2.3181,5.937660


In [310]:
best_scn2_df = summarize_best_by_column(tdf, '010_Yield')
best_scn2_df.sort_values(by=['Rating'])

,Ticker,Spread,Type,Mult,Rating,Price,AnnualDiv,CurYieldPct,MSpread,030_Yield,050_Yield,010_Yield,ExpYield
29,SLF.PR.K,217.0,T,NaN,P2H,23.30,0.5882,2.5245,2.3415,2.8544,3.3447,2.2837,2.491745
24,IFC.PR.D,266.0,T,NaN,P2I,24.87,0.7107,2.8577,2.6747,2.8836,2.9149,2.8478,2.455305
32,TRP.PR.F,192.0,T,NaN,P2L,17.00,0.5257,3.0924,2.9094,4.8156,7.4750,1.8027,4.456820
18,FFH.PR.D,315.0,T,NaN,P3H,20.80,0.8332,4.0058,3.8228,4.5839,5.4914,3.5839,4.023190
12,BPO.PR.S,348.0,T,NaN,P3I,19.75,0.9157,4.6365,4.4535,5.2951,6.3418,4.1565,4.648650
0,ALA.PR.B,266.0,T,NaN,P3L,18.64,0.7107,3.8128,3.6298,4.8346,6.4375,3.0598,4.332655


In [301]:
best_scn1_df = summarize_best_by_column(tdf, '030_Yield')
best_scn1_df.sort_values(by=['Rating'])

,Ticker,Spread,Type,Mult,Rating,Price,AnnualDiv,CurYieldPct,MSpread,030_Yield,050_Yield,010_Yield,ExpYield,018_Yield
28,SLF.PR.J,141.0,T,NaN,P2H,15.42,0.3982,2.5824,2.3994,5.2857,9.3847,0.5306,5.042085,0.5306
25,MFC.PR.P,141.0,T,NaN,P2I,16.75,0.3982,2.3773,2.1943,4.7002,8.1839,0.6091,4.456985,0.6091
33,TRP.PR.H,128.0,T,NaN,P2L,12.85,0.3658,2.8467,2.6637,6.5660,12.3082,-0.0073,6.359320,-0.0073
19,FFH.PR.F,216.0,T,NaN,P3H,16.06,0.5857,3.6469,3.4639,5.3869,8.1078,2.3606,4.956460,2.3606
22,FN.PR.B,207.0,T,NaN,P3I,13.32,0.5632,4.2282,4.0452,6.5986,10.3528,2.4822,6.135720,2.4822
30,TA.PR.E,203.0,T,NaN,P3L,13.68,0.5532,4.0439,3.8609,6.3785,10.0664,2.3181,5.937660,2.3181


In [302]:
best_scn2_df = summarize_best_by_column(tdf, '050_Yield')
best_scn2_df.sort_values(by=['Rating'])

,Ticker,Spread,Type,Mult,Rating,Price,AnnualDiv,CurYieldPct,MSpread,030_Yield,050_Yield,010_Yield,ExpYield,018_Yield
28,SLF.PR.J,141.0,T,NaN,P2H,15.42,0.3982,2.5824,2.3994,5.2857,9.3847,0.5306,5.042085,0.5306
25,MFC.PR.P,141.0,T,NaN,P2I,16.75,0.3982,2.3773,2.1943,4.7002,8.1839,0.6091,4.456985,0.6091
33,TRP.PR.H,128.0,T,NaN,P2L,12.85,0.3658,2.8467,2.6637,6.5660,12.3082,-0.0073,6.359320,-0.0073
23,FTS.PR.I,145.0,T,NaN,P3H,15.35,0.4083,2.6599,2.4769,5.2982,9.3446,0.6238,5.047630,0.6238
22,FN.PR.B,207.0,T,NaN,P3I,13.32,0.5632,4.2282,4.0452,6.5986,10.3528,2.4822,6.135720,2.4822
30,TA.PR.E,203.0,T,NaN,P3L,13.68,0.5532,4.0439,3.8609,6.3785,10.0664,2.3181,5.937660,2.3181


In [33]:
'foo' + 'bar'

'foobar'

In [35]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-08-18,24.139999,24.24,24.129999,24.129999,24.129999,1300


In [40]:
for i in data.index:
    print(i)
    print(data.at[i,'Close'])

2021-08-18 00:00:00
24.1299991607666


In [44]:
row1=data.iloc[0]['Close']
print(round(row1,2))

24.13


In [ ]:
tdf.rename(columns={"Close":"Price"},errors="ignore",inplace=True)
tdf